In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv("tweets_2020-04-30.csv")

In [4]:
data

,Gender,Mood,text,description,screenName,Unnamed: 5,favorited,favoriteCount,replyToSN,created.x,...,name,created.y,protected,verified,location,lang,id.y,listedCount,followRequestSent,profileImageUrl
0,m,-1,"@econoclasta Profe, no se olvide de los 3 zomb...",Actions Speak Louder than Words,A_LaraV,1,False,0,econoclasta,2020-04-30 18:02:03,...,Antoine C,2010-08-30 2:34:05,False,False,Barcelona,NaN,1.846357e+08,0,False,http://pbs.twimg.com/profile_images/1518416012...
1,m,1,@zoriviajero @preferredhotels @OFFPOLINN @ACAV...,Crónica de viajes | Periodismo cultural | Edit...,a_todaletra,2,False,1,zoriviajero,2020-04-30 18:01:03,...,josé alejandro,2011-11-16 15:41:15,False,False,"Barcelona, España",NaN,4.140591e+08,38,False,http://pbs.twimg.com/profile_images/1221104390...
2,m,0,¡Echa un vistazo a mi transmisión desde mi Pla...,NaN,abrahamcruz0726,3,False,0,NaN,2020-04-30 18:02:09,...,Abrahamcruz0726,2018-08-17 19:45:13,False,False,"Barcelona, España",NaN,1.030541e+18,0,False,http://pbs.twimg.com/profile_images/1106306643...
3,F,0,hoy tengo una cita sólo para hacer lo mismo de...,ditto 🖇🖇🖇🖇🖇🖇 https://t.co/pEcO20y9Vf w/@adaman...,aerinita,4,False,0,NaN,2020-04-30 18:00:33,...,iren,2011-09-05 13:55:31,False,False,bcn,NaN,3.683462e+08,6,False,http://pbs.twimg.com/profile_images/1246484954...
4,m,1,#quedateencasa\nSi tienes jardín y piscina en ...,"CEO - Diseño, creación y conservación de #PISC...",AivanFerron,5,False,0,NaN,2020-04-30 18:03:27,...,Aivan Ferron,2011-12-21 8:17:38,False,False,Barcelona,NaN,4.425963e+08,8,False,http://pbs.twimg.com/profile_images/1075647865...
5,m,0,Llevo todo el día con el remix de Savage en re...,ready • https://t.co/iEzEKVhM29,AlexPasalodos,6,False,0,NaN,2020-04-30 18:01:55,...,alex,2012-11-04 19:30:42,False,False,Barcelona,NaN,9.259938e+08,25,False,http://pbs.twimg.com/profile_images/1214311174...
6,f,-1,@Hastalappk Yo llevo dos días con que quiero a...,Animales🐾\n\nLa guitarra suena un poco mal per...,ana_sgd,7,False,1,Hastalappk,2020-04-30 18:02:17,...,AnnaChop (última vez ✈️💀 en misa),2010-07-24 12:51:44,False,False,Barcelona,NaN,1.702949e+08,6,False,http://pbs.twimg.com/profile_images/1230583148...
7,i,1,En @AnfapaNews nos unimos al aplauso solidario...,ANFAPA es la Asociación Nacional de Fabricante...,AnfapaNews,8,False,0,NaN,2020-04-30 18:00:53,...,Anfapa-MorterosySATE,2015-02-06 10:32:03,False,False,"Barcelona, Cataluña",NaN,3.021230e+09,35,False,http://pbs.twimg.com/profile_images/9761850216...
8,i,1,💙 @FundlaCaixa dona millón de euros más a la c...,El Banc dels Aliments de Barcelona té com a pr...,BancAlimentsBcn,9,False,0,NaN,2020-04-30 18:00:01,...,Banc dels Aliments,2011-09-14 8:29:56,False,False,Barcelona,NaN,3.732630e+08,180,False,http://pbs.twimg.com/profile_images/3788000006...
9,i,1,🔔 Cómo proteger el planeta cuando sales a come...,#Curiosidades #Noticias #Actualidad Compartimo...,BCN_0N,10,False,0,NaN,2020-04-30 18:00:26,...,BCN 🔛,2015-04-03 8:21:25,False,False,"Barcelona, España",NaN,3.131589e+09,40,False,http://pbs.twimg.com/profile_images/9858485581...


In [5]:
#Processing the text to be used for training and testing of the model
processed_data = []
for i in range(0, len(data)):
    processed_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', str(data['text'][i]))
    processed_text = re.sub("(@[A-Za-z0-9_]+)","", processed_text)
    processed_text = re.sub(r'\W', ' ', processed_text)
    processed_text = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_text)
    processed_text = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_text) 
    processed_text = re.sub(r'\s+', ' ', processed_text, flags=re.I)
    processed_text = processed_text.lower()
    processed_data.append(processed_text)

In [74]:
len(processed_data)

123

In [6]:
cv = CountVectorizer()

In [7]:
cv.fit(processed_data)
X = cv.transform(processed_data) #transformed data into numeric representation
X.shape
y = data["Mood"]

In [8]:
#Normalization of the data

data_normalized = preprocessing.normalize(X)

In [11]:
skf = StratifiedKFold(n_splits=5)
print(skf)

accuracies = []
for train_index, test_index in skf.split(data_normalized, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = data_normalized[train_index], data_normalized[test_index]
    y_train, y_test = y[train_index], y[test_index]
    classifier = LinearSVC()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
#     print("pred", y_pred)
#     print("real", y_test.to_numpy())
    acc = classifier.score(X_test, y_test)
    accuracies.append(acc)


StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


In [12]:
# X_train, X_text, y_train, y_test = train_test_split(X, data["Mood"], test_size=0.2, random_state=0)
print(accuracies)

[0.5384615384615384, 0.38461538461538464, 0.6, 0.391304347826087, 0.5652173913043478]


In [15]:
np.count_nonzero(data["Mood"].to_numpy()==1)

23

In [29]:
from sklearn.neighbors import KNeighborsClassifier

class Predictor:
    def __init__(self, datafile, classifier):
        self.data = pd.read_csv(datafile)
        self.classifier = classifier #LinearSVC()
    
    def clean_data(self):
        self.processed_data = []
        for i in range(0, len(self.data)):
            processed_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', str(self.data['text'][i]))
            processed_text = re.sub("(@[A-Za-z0-9_]+)","", processed_text)
            processed_text = re.sub(r'\W', ' ', processed_text)
            processed_text = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_text)
            processed_text = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_text) 
            processed_text = re.sub(r'\s+', ' ', processed_text, flags=re.I)
            processed_text = processed_text.lower()
            self.processed_data.append(processed_text)
        return self.processed_data
    
    
    def transform_normalize(self, processed_data):
        cv = CountVectorizer()
        cv.fit(processed_data)
        X = cv.transform(processed_data) #transformed data into numeric representation
        self.data_normalized = preprocessing.normalize(X)
        return self.data_normalized
    
    def split(self, data_normalized, test_size):
        X = data_normalized
        y = np.asarray(self.data["Mood"])
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size = test_size, random_state = 3)

    def fit(self):
        self.predictor = self.classifier.fit(self.X_train, self.y_train)
        
    def train_model(self, data_normalized, test_size):
        self.split(data_normalized, test_size)
        self.fit()
        return self.predictor
    
    def predict(self, input_value):
        if input_value == None:
            result = self.classifier.predict(self.X_test)
        else:
            result = self.classifier.predict(np.array([input_values]))
        return result

    

In [34]:
if __name__ == '__main__':
    predictor_model = Predictor("tweets_2020-04-30.csv", KNeighborsClassifier())
    predictor_model.clean_data()
    predictor_model.transform_normalize(predictor_model.processed_data)
    pred = predictor_model.train_model(predictor_model.data_normalized, 0.2)
#     predictor_model.split(predictor_model.data_normalized, 0.2)
#     predictor_model.fit()
#     print("Accuracy: ", predictor_model.predictor.score(predictor_model.X_test, predictor_model.y_test))
#     print(predictor_model.predict(None))
    print(pred)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


In [15]:
if hasattr(predictor_model, 'X_train') and hasattr(predictor_model, 'X_test') and hasattr(predictor_model, 'y_train') and hasattr(predictor_model, 'y_test'):
    correct = True
else:
    correct = False
correct

True

In [64]:
bool(False)

False

In [60]:
import unittest

class PredictorTest(unittest.TestCase):
    def setUp(self):
        self.predictor = Predictor("tweets_2020-04-30.csv")
        self.predictor.clean_data()
        
    def test_data_is_cleaned(self):
        self.assertNotEqual(self.predictor.data['text'], self.predictor.processed_data)
    
    def test_transform_normalize(self):
        self.predictor.transform_normalize()
        self.assertNotEqual(self.predictor.processed_data, self.predictor.data_normalized)
    
    def test_split():
        self.predictor.split(0.2)
        if hasattr(self.predictor, 'X_train') and hasattr(self.predictor, 'X_test') and hasattr(self.predictor, 'y_train') and hasattr(self.predictor, 'y_test'):
            correct = True
        else:
            correct = False
        self.assertTrue(correct)
        
    
    